In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import re 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.impute import SimpleImputer

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
import spacy

In [2]:
data = pd.read_csv("final_merged_data.csv")
print(data.shape)
data = data.dropna(axis = 0, how = 'any')
print(data.shape)

(44401, 7)
(43371, 7)


In [3]:
## Grab a corpus of text as a numpy array instead of series
text = data.questions_body.values.astype('U')
text

array(['sophomore jobs field nurse fun day years schooling medicine babies ',
       'companies ',
       'hi sophomore employers college degree chance job university school college jobs employers hire ',
       ..., 'collage singer collage singer ',
       'mcat semester study time time-management ',
       'auto careers mechanics '], dtype='<U3906')

In [4]:
token = RegexpTokenizer(r'[a-z]+')

In [5]:
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1), tokenizer = token.tokenize)
text_counts = cv.fit_transform(text)
text_counts = text_counts.toarray()
vocab = cv.get_feature_names()
text_counts = pd.DataFrame(text_counts, columns=vocab) ## yields pandas dataframe
text_counts.sample(3) 


,aa,aaa,aadsas,aand,aant,aauw,ab,aba,abc,abduction,...,zip,zombie,zone,zoo,zookeeper,zookeepers,zoologist,zoology,zoos,zymeria
24698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6374,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Hm, some weird words here and there that remains after preprocessing. 

In [6]:
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(text)
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names()

## eliminate vocab beginning with numbers
#vocab = [w for w in vocab if re.search(r'[a-z]', w)]
tfDF = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab) ## also yields pandas dataframe
tfDF.sample(3)

,12,12629,12education,12th,14,16,1872,22,23,24,...,zombie,zone,zoo,zookeeper,zookeepers,zoologist,zoology,zoos,zymeria,áreas
10226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Hm, more weird stuff that remains after preprocessing according to this method

In [7]:
temp = pd.merge(tfDF, data, left_index = True, right_index = True)

In [8]:
temp.shape
temp.sample(2)

,12,12629,12education,12th,14,16,1872,22,23,24,...,zoos,zymeria,áreas,professionals_id,professionals_headline,professionals_industry,answers_question_id,questions_id,questions_title,questions_body
16206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,e963b61ee35c47c49551d0b3eee97782,Client Support Eng,Dell,9565d31d2de84b7a915aa4aaff35097c,9565d31d2de84b7a915aa4aaff35097c,career path court judge justice,career law justice judge social-justice
14657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3a3714f743554c57ba6e1c110add2d65,Zoo Keeper & Pet/House Sitter (Owner),Zoo keeping,8b0db74584e2439e8b3e2631e196a836,8b0db74584e2439e8b3e2631e196a836,college student,graduate bachelors masters courses ones applic...


In [9]:
X = tfDF
y = data['professionals_headline']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=2)
#imp = SimpleImputer(missing_values=np.nan, strategy='median')
#X = imp.fit_transform(X)


In [ ]:
knn = KNeighborsClassifier(n_neighbors=31, weights='distance')
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)